<div style="background-color:yellow; text-align:center; text-align:center; padding:40px;">
<h1  style="color:red;" > MMI-714 : Generative Models for Multimedia </h1>   
<h2  style="color:red;" > Final Report </h2>
<br>
<h3  style="color:red;  font-style:italic;" > Exploration of the Intuitive Physics through the 
Latent Space Disentanglement</h3>
<br>
<h4  style="color:red;" >Turgay Yıldız</h4>
<br>
<h4  style="color:red;" >Cognitive Sciences,  Middle East Technical University (METU)</h4>
</div>


<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  First Import the Relevant Packages  </h2>
</div>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.checkpoint import checkpoint 
from torch.amp import GradScaler, autocast
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
from torch.nn.functional import relu
import torch.nn.init as init
from torchvision import models
from torch.amp import GradScaler, autocast
from PIL import Image
from einops.layers.torch import Rearrange
from einops import repeat
from torch import Tensor
import itertools
import tqdm
import pandas as pd
import scipy as sp
import cv2
import os 
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Load the Dataset  </h2>
</div>

In [ ]:
path = "/home/turgay/Turgay/Academic/2024-2025/Fall/Generative_Models/Final_Project/Data/6_pairs/" 

In [ ]:
path_data = path + "concatenated_data.npy"

In [ ]:
X = np.load(path_data, mmap_mode="r")

In [ ]:
X.shape

In [ ]:
X  =  X.reshape(3529, 6, 3, 224, 224)

In [ ]:
X_image =  X[:, 0, :, :, :] 
X_color =  X[:, 1, :, :, :] 
X_order =  X[:, 2, :, :, :] 

X_image2 =  X[:, 3, :, :, :]
X_color2 =  X[:, 4, :, :, :]
X_order2 =  X[:, 5, :, :, :]

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, X):
        
        self.X     =    X
 
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        
        img_array   = np.array(self.X[idx], dtype=np.float32)       
        img_tensor  = torch.tensor(img_array)                           


        return img_tensor

In [ ]:
dataset = ImageDataset(X_image)

In [ ]:
indices = list(range(len(dataset)))

train_indices, val_indices = train_test_split(indices, test_size=0.2, shuffle=False)

train_dataset = Subset(dataset, train_indices)
val_dataset   = Subset(dataset, val_indices)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=True)

In [ ]:
for i in train_loader:
    print(i.shape)
    break

In [ ]:
i  =  i.reshape(-1, 224, 224, 3)

In [ ]:
i.min(), i.max()

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Function to Plot Images :  </h2>
</div>

In [ ]:
def plot_img(data, row, col, size1, size2, c_map = None): 
    
    if (row == 1) and (col == 1): 
        fig, ax = plt.subplots(1, 1, figsize=(size1, size2))
        ax.imshow(data, cmap=c_map) 
        ax.set_axis_off() 

    elif (row == 1) and (col > 1):
        fig, axes = plt.subplots(1, col, figsize=(size1, size2))
        for i in range(col):
            axes[i].imshow(data[i], cmap=c_map) 
            axes[i].set_axis_off() 
            axes[i].set_title(f"Image {i}")

    else:
        fig, axes = plt.subplots(row, col, figsize=(size1, size2))
        axes = axes.flatten() 
        for i in range(row * col):
            if i < len(data):  
                axes[i].imshow(data[i], cmap=c_map) 
                axes[i].set_axis_off()  
                axes[i].set_title(f"Image {i}")
            else:
                axes[i].axis('off') 

    plt.tight_layout()  
    plt.show()


In [ ]:
X_image.shape

In [ ]:
plot_img(i[:12] / 255., 1, 12, 15, 15) 

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Model </h2>
</div>

In [ ]:
import torch
import torch.nn as nn

class Beta_VAE(nn.Module):
    def __init__(self, inp_out_dim=0, latent_dim=0, first_dim=0, dropout_prob=0.2):
        super(Beta_VAE, self).__init__()

        self.latent_dim = latent_dim
        self.inp_out_dim = inp_out_dim
        self.first_dim = first_dim
        self.dropout_prob = dropout_prob

        #----------------------------------------------------------------------------------------------------------------------------#
        #-------------------------------------------           ENCODER               ------------------------------------------------#
        #----------------------------------------------------------------------------------------------------------------------------#

        self.cnn_encoder = nn.Sequential(
            # Block 1
            nn.Conv2d(self.inp_out_dim, self.first_dim, kernel_size=7, padding=3),
            nn.BatchNorm2d(self.first_dim),
            nn.LeakyReLU(),
            nn.Conv2d(self.first_dim, self.first_dim, kernel_size=7, padding=3),
            nn.BatchNorm2d(self.first_dim),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 112

            # Block 2
            nn.Conv2d(self.first_dim, self.first_dim * 2, kernel_size=5, padding=2),
            nn.BatchNorm2d(self.first_dim * 2),
            nn.LeakyReLU(),
            nn.Conv2d(self.first_dim * 2, self.first_dim * 2, kernel_size=5, padding=2),
            nn.BatchNorm2d(self.first_dim * 2),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 56

            # Block 3
            nn.Conv2d(self.first_dim * 2, self.first_dim * 4, kernel_size=5, padding=2),
            nn.BatchNorm2d(self.first_dim * 4),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 28

            # Block 4
            nn.Conv2d(self.first_dim * 4, self.first_dim * 8, kernel_size=5, padding=2),
            nn.BatchNorm2d(self.first_dim * 8),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 14

            # Block 5
            nn.Conv2d(self.first_dim * 8, self.first_dim * 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(self.first_dim * 16),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 7

            # Block 6
            nn.Conv2d(self.first_dim * 16, self.first_dim * 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(self.first_dim * 32),
            nn.LeakyReLU(),
            nn.Flatten()
        )

        #----------------------------------------------------------------------------------------------------------------------------#
        #-------------------------------------------           FC_MU & LOG           ------------------------------------------------#
        #----------------------------------------------------------------------------------------------------------------------------#

        self.fc_mu = nn.Sequential(
            
            nn.Linear(self.first_dim * 32 * 7 * 7, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(256, self.latent_dim),
        )

        self.fc_logvar = nn.Sequential(
            
            nn.Linear(self.first_dim * 32 * 7 * 7, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(256, self.latent_dim),
        )

        #----------------------------------------------------------------------------------------------------------------------------#
        #-------------------------------------------           DECODER               ------------------------------------------------#
        #----------------------------------------------------------------------------------------------------------------------------#

        self.fc_decoder = nn.Sequential(
            
            nn.Linear(self.latent_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Dropout(self.dropout_prob),

            nn.Linear(1024, self.first_dim * 128 * 7 * 7),
            nn.BatchNorm1d(self.first_dim * 128 * 7 * 7),
            nn.LeakyReLU()
        )

        self.cnn_decoder = nn.Sequential(
            
            nn.ConvTranspose2d(self.first_dim * 128, self.first_dim * 64, kernel_size=4, stride=2, padding=1),  # 7 -> 14
            nn.BatchNorm2d(self.first_dim * 64),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(self.first_dim * 64, self.first_dim * 32, kernel_size=4, stride=2, padding=1),  # 14 -> 28
            nn.BatchNorm2d(self.first_dim * 32),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(self.first_dim * 32, self.first_dim * 16, kernel_size=4, stride=2, padding=1),  # 28 -> 56
            nn.BatchNorm2d(self.first_dim * 16),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(self.first_dim * 16, self.first_dim * 8, kernel_size=4, stride=2, padding=1),  # 56 -> 112
            nn.BatchNorm2d(self.first_dim * 8),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(self.first_dim * 8, self.first_dim * 2, kernel_size=4, stride=2, padding=1),  # 112 -> 224
            nn.BatchNorm2d(self.first_dim * 2),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(self.first_dim * 2, self.inp_out_dim, kernel_size=5, stride=1, padding=2),
            #nn.Sigmoid()
        )

        # Initialize weights
        self.apply(self.init_weights)

    #----------------------------------------------------------------------------------------------------------------------------#
    #-------------------------------------------           FUNCTIONS               ----------------------------------------------#
    #----------------------------------------------------------------------------------------------------------------------------#

    @staticmethod
    def init_weights(m):
        """
        Initialize weights for linear and convolutional layers using Kaiming initialization.
        """
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def encode(self, x):
        h = self.cnn_encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.fc_decoder(z).reshape(-1, self.first_dim * 128, 7, 7)  # Use reshape instead of view
        return self.cnn_decoder(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z)

        kl_loss = -0.5 * torch.sum(1 + logvar - mu**2 - torch.exp(logvar), dim=-1)
        return reconstructed, kl_loss.mean()

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  To Employ GPU </h2>
</div>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Move Model to GPU  </h2>
</div>

In [ ]:
model   =   Beta_VAE(inp_out_dim = 3, latent_dim=16, first_dim = 4, dropout_prob = 0.2).to(device) 

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Number of trainable parameters in the model:", num_params)

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  To Save Weights and Losses : </h2>
</div>

In [ ]:
path_model   =  "/home/turgay/Turgay/Academic/2024-2025/Fall/Generative_Models/Final_Project/Final_Report/codes/weights/Beta_VAE/Beta_VAE_weights_1.pth"
path_losses  =  "/home/turgay/Turgay/Academic/2024-2025/Fall/Generative_Models/Final_Project/Final_Report/codes/weights/Beta_VAE/Beta_VAE_losses_1.pth"

In [ ]:
#torch.save({
            'model_state_dict': model.state_dict(),
            'best_loss'       : 9999999999999,
        }, path_model)   

In [ ]:
#torch.save({
            'train_loss_rec': [],
            'train_loss_kl' : [],
            'train_loss'    : [],
    
            'val_loss_rec'  : [],
            'val_loss_kl'   : [],
            'val_loss'      : [],
    
            'epochs'        : [],
    
        }, path_losses)    

In [ ]:
checkpoint_weights   =   torch.load(path_model, weights_only=True) 
checkpoint_losses    =   torch.load(path_losses, weights_only=True)  

model.load_state_dict(checkpoint_weights['model_state_dict'])

best_loss       =  checkpoint_weights['best_loss'] 

train_loss      =  checkpoint_losses['train_loss']
train_loss_kl   =  checkpoint_losses['train_loss_kl']
train_loss_rec  =  checkpoint_losses['train_loss_rec']

val_loss        =  checkpoint_losses['val_loss'] 
val_loss_kl     =  checkpoint_losses['val_loss_kl'] 
val_loss_rec    =  checkpoint_losses['val_loss_rec'] 

epochs          =  checkpoint_losses['epochs'] 

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Parameters :  </h2>
</div>

In [ ]:
#------------------------------------
beta_start    =   0.0001
beta_end      =   50

num_epochs    =   100
#------------------------------------
mu_values     =   [] 
logvar_values =   []

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Schedulers for Beta </h2>
</div>

In [ ]:
def linear_schedule(x, start=0, end=10):
    return start + (end - start) * (x / x.max())
#---------------------------------------------------------------------------------#
def exponential_schedule(epoch):
    return beta_start * ((beta_end / beta_start) ** (epoch / num_epochs))
#---------------------------------------------------------------------------------#
def exponential_schedule_smooth(epoch=0, num_epochs=100, x_start=1, x_end=100):
    # Interpolation factor using a sigmoid curve to ensure a smoother transition
    scale = 1 / (1 + np.exp(-10 * (epoch / num_epochs - 0.5)))  # Sigmoid curve (0 to 1)
    return x_start * (x_end / x_start) ** scale

In [ ]:
plt.plot(np.arange(0, 1000, 1), exponential_schedule_smooth(np.arange(0, 1000, 1), 1000, 1, 1000) / 20, "r.") 

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   Train and Validation Function </h2>
</div>

In [ ]:
def train(num_epochs=10, beta=0, patience=10, counter=0, learning_rate=0.001, best_loss=best_loss):
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
    criterion = nn.MSELoss()
    scaler    = GradScaler()

    for epoch in range(num_epochs):
        
        beta  =  exponential_schedule_smooth(epoch=epoch, num_epochs=250, x_start=1, x_end=250) / 5
        
        model.train()

        total_loss_train      =   0.0
        total_loss_train_kl   =   0.0
        total_loss_train_rec  =   0.0

        for batch in tqdm.tqdm(train_loader):
            
            images = batch.to(device)

            optimizer.zero_grad()

            with autocast(device_type='cuda'):
                
                img_recon, kl_loss = model(images)
                img_rec_loss       = criterion(img_recon, images)
                loss               = img_rec_loss + kl_loss * beta

            scaler.scale(loss).backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  
            scaler.step(optimizer)
            scaler.update()

            total_loss_train     += loss.item()
            total_loss_train_kl  += kl_loss.item()
            total_loss_train_rec += img_rec_loss.item()

        total_loss_train     /= len(train_loader)
        total_loss_train_kl  /= len(train_loader)
        total_loss_train_rec /= len(train_loader)

        train_loss.append(total_loss_train)
        train_loss_kl.append(total_loss_train_kl)
        train_loss_rec.append(total_loss_train_rec)

        if torch.isnan(torch.tensor(total_loss_train)):
            print("NaN value encountered!")
            break

        print("-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]          |        Total Train Loss : {total_loss_train:.4f}           |")
        print("-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]          |        Total KL Loss : {total_loss_train_kl:.4f}              |")
        print("-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]          |        Total REC Loss : {total_loss_train_rec:.4f}              |")
        print("-------------------------------------------------------------------------------")

        model.eval()

        total_loss_val      =   0.0
        total_loss_val_kl   =   0.0
        total_loss_val_rec  =   0.0

        with torch.no_grad():
            for batch in tqdm.tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                images = batch.to(device)

                img_recon, kl_loss = model(images)
                img_rec_loss       = criterion(img_recon, images)
                loss               = img_rec_loss + kl_loss * beta

                total_loss_val     += loss.item()
                total_loss_val_kl  += kl_loss.item()
                total_loss_val_rec += img_rec_loss.item()

        total_loss_val     /= len(val_loader)
        total_loss_val_kl  /= len(val_loader)
        total_loss_val_rec /= len(val_loader)

        val_loss.append(total_loss_val)
        val_loss_kl.append(total_loss_val_kl)
        val_loss_rec.append(total_loss_val_rec)

        print("-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]         |       Total Validation Loss : {total_loss_val:.4f}           |")
        print("-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]         |       Total KL Loss : {total_loss_val_kl:.4f}              |")
        print("-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]         |       Total REC Loss : {total_loss_val_rec:.4f}             |")
        print("-------------------------------------------------------------------------------")

        if len(val_loss) >= 2:
            res = ((val_loss[-2] - val_loss[-1]) / val_loss[-2]) * 100
            print("-------------------------------------------------------------------------------")
            print(f"|              Change in loss is      %   {res:.2f}                               |")
            print("-------------------------------------------------------------------------------")

        if  total_loss_val_rec < best_loss:
            
            print("*************...saving best model *************")
            
            best_loss = total_loss_val_rec
            
            torch.save({
                'model_state_dict': model.state_dict(),
                'best_loss'       : best_loss,
            }, path_model)
             

        epochs.append(epoch)

        torch.save({
            'train_loss_rec': train_loss_rec,
            'train_loss_kl': train_loss_kl,
            'train_loss': train_loss,
            'val_loss_rec': val_loss_rec,
            'val_loss_kl': val_loss_kl,
            'val_loss': val_loss,
            'epochs': epochs,
        }, path_losses)

        if (len(val_loss) >= 2) and (val_loss[-2] > val_loss[-1]):
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping!")
                break

        scheduler.step()  
        torch.cuda.empty_cache()

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   Start Train and Validation </h2>
</div>

In [ ]:
train(num_epochs=250, beta=0, patience=10, counter=0, learning_rate=0.001 ,  best_loss=550)   

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  To Plot Train and Validation Losses </h2>
</div>

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,6))

ax.plot(train_loss_kl, "b-", label="Train Loss KL")
ax.plot(val_loss_kl, "r-", label="Validation Loss KL")
ax.set_xlabel("Epochs")
ax.set_ylabel("KL_Loss")
ax.legend()

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   To Test Reconstruction Quality </h2>
</div>

In [ ]:
with torch.no_grad():
    for img in val_loader:
        img    = img.to(device)

        recon, kl  = model(img)
        break

In [ ]:
img.shape, recon.shape

In [ ]:
img.max(), recon.min(), recon.max()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(dpi=250)

fig, ax = plt.subplots(2, 7, figsize=(15, 4))

for i in range(7):
	ax[0, i].imshow( (img[i]/255.).reshape(224,224,3).cpu().numpy())
	ax[1, i].imshow( (recon[i]/255.).reshape(224, 224, 3).cpu().numpy())
	ax[0, i].axis('OFF')
	ax[1, i].axis('OFF')
plt.show()


<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   Test Latent Space </h2>
</div>

In [ ]:
e = model.eval()

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:1px;">
<h2>   The following image will be sent to the latent space: </h2>
</div>

In [ ]:

plt.figure(dpi=250)

fig, ax = plt.subplots(1, 1, figsize=(4, 4))

ax.imshow((img[5]/255.).reshape(224,224,3).cpu().numpy())

plt.show()


<div span style="background-color:yellow;    color:red;      text-align:center;    padding:1px;">
<h2>   Create Vector z : </h2>
</div>

In [ ]:
mu, logvar = model.encode(img[5].reshape(1, 3, 224, 224))

In [ ]:
z = model.reparameterize(mu, logvar)

In [ ]:
z.shape

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:1px;">
<h2>   Traversals Function : </h2>
</div>

In [ ]:
def latent_space_traversal(model, z, latent_dim, steps=10, range_val=3):

    device = next(model.parameters()).device  

    fig, axs = plt.subplots(latent_dim, steps, figsize=(steps * 2, latent_dim * 2))

    for dim in range(latent_dim):
        for step, val in enumerate(torch.linspace(-range_val, range_val, steps)):
 
            z_traversal             =   z.clone()
            z_traversal[0, dim]     =   val 

            with torch.no_grad():
                
                recon              = model.decode(z_traversal).squeeze(0).cpu().numpy()

            axs[dim, step].imshow((recon/255.).reshape(224, 224, 3))
            axs[dim, step].axis("off") 

In [ ]:
latent_space_traversal(model, z, 16) 